<a href="https://colab.research.google.com/github/krdeepak39/Data-Scrapping-project/blob/main/Scrapping_list_of_college__Canada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import re
import pandas as pd
def get_info_box(url):
    # Send a request to the URL
    response = requests.get(url)

    # Parse the content using BeautifulSoup
    soup = bs(response.content, 'html.parser')

    # Find the infobox table with class 'infobox vcard'
    info_box = soup.find('table', {'class': 'infobox vcard'})

    # Dictionary to hold the extracted information
    info = {}

    if info_box:
        # Extract rows from the infobox table
        rows = info_box.find_all('tr')

        for row in rows:
            # Get the header (th) and data (td) from each row
            header = row.find('th')
            data = row.find('td')

            if header and data:
                # Get text from header and data, stripping extra spaces
                key = header.get_text(" ", strip=True)
                value = data.get_text(" ", strip=True)

                # Clean the value
                value = value.replace('\n', ' ')
                value = re.sub(r'\[.*?\]', '', value)  # Remove content within brackets
                value = re.sub(r'\s+', ' ', value).strip()  # Remove extra spaces

                # Store the cleaned key-value pair in the dictionary
                info[key] = value

    return info

# Example usage
url = 'https://en.wikipedia.org/wiki/Alberta_University_of_the_Arts'
college_info = get_info_box(url)
print(college_info)


{'Former names': 'Alberta College of Art Alberta College of Art and Design', 'Type': 'Public', 'Established': '1926 ( 1926 )', 'President': 'Daniel Doz', 'Academic staff': '145', 'Administrative staff': '95', 'Students': '1,323 as of fall 2012', 'Address': '1407 14 Avenue NW Calgary , Alberta T2N 4R3 51°03′43″N 114°05′29″W \ufeff / \ufeff 51.06205°N 114.09143°W \ufeff / 51.06205; -114.09143 \ufeff ( Alberta University of the Arts )', 'Campus': 'Urban', 'Colours': 'Red White', 'Affiliations': 'AICAD , ACCC , CCAA , Alberta Colleges Athletics Conference , Alberta Association of Colleges & Technical Institutes , CBIE, NASAD', 'Website': 'auarts .ca', 'ASN': '54307'}


In [2]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re

def clean_text(text):
    if isinstance(text, str):
        text = re.sub(r'\n', ' ', text)  # Remove newlines
        text = re.sub(r'\[', '', text)  # Remove opening square brackets
        text = re.sub(r'\]', '', text)  # Remove closing square brackets
        text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
        text = text.replace('\ufeff', '')  # Remove unicode character
        text = text.replace(', ,', ',')  # Remove extra commas
    return text

def clean_address(text):
    if isinstance(text, str):
        text = re.sub(r'\d{2,3}°\d{2}′\d{2}″[N|S|E|W]', '', text)  # Remove latitude and longitude
        text = re.sub(r'\(.*?\)', '', text)  # Remove content within parentheses
        text = text.replace(' / ', '').strip()  # Remove ' / ' and strip spaces
    return text

def get_info_box(url):
    # Send a request to the URL
    response = requests.get(url)

    # Parse the content using BeautifulSoup
    soup = bs(response.content, 'html.parser')

    # Find the infobox table with class 'infobox vcard'
    info_box = soup.find('table', {'class': 'infobox vcard'})

    # Dictionary to hold the extracted information
    info = {}

    if info_box:
        # Extract rows from the infobox table
        rows = info_box.find_all('tr')

        for row in rows:
            # Get the header (th) and data (td) from each row
            header = row.find('th')
            data = row.find('td')

            if header and data:
                # Get text from header and data, stripping extra spaces
                key = header.get_text(" ", strip=True)
                value = data.get_text(" ", strip=True)

                # Clean the value
                value = clean_text(value)

                # Store the cleaned key-value pair in the dictionary
                info[key] = value

    return info

def save_data_to_csv(title, data):
    df = pd.DataFrame(data)
    # Ensure the 'name' column is the first column
    columns = ['Name'] + [col for col in df.columns if col != 'Name']
    df = df[columns]
    df.to_csv(title, index=False, encoding='utf-8')

# Fetch the webpage
response = requests.get("https://en.wikipedia.org/wiki/List_of_universities_in_Canada")
soup = bs(response.content, 'html.parser')

# Select only the first <a> tag within each <tr> tag of the first table with class 'wikitable sortable'
rows = soup.select(".wikitable.sortable tr")
college_links = [row.select_one("td:nth-of-type(1) a") for row in rows if row.select_one("td:nth-of-type(1) a")]

# Base URL for Wikipedia
base_url = 'https://en.wikipedia.org'

# List to store college information
college_info_list = []

# Loop through the college links
for index, college in enumerate(college_links):
    # Debugging line to break after 10 iterations, uncomment if needed
    # if index == 10:
    #     break

    # Print progress every 10 colleges
    if index % 10 == 0:
        print(f'Processing college {index}: {college.get_text()}')

    try:
        # Construct the full URL for each college
        relative_url = college['href']
        full_url = base_url + relative_url
        title = college['title']

        # Retrieve information box for each college and append to the list
        college_info = get_info_box(full_url)
        college_info['Name'] = title
        college_info_list.append(college_info)

    except Exception as e:
        # Print the college name and the error message
        print(f'Error processing college {college.get_text()}: {e}')

# Save the college information to a CSV file
save_data_to_csv('University_info.csv', college_info_list)


Processing college 0: Alberta University of the Arts
Processing college 10: Concordia University
Processing college 20: Royal Roads University
Processing college 30: Université de Saint-Boniface
Processing college 40: Saint Mary's University
Processing college 50: Trent University
Processing college 60: University of Windsor
Processing college 70: Université du Québec en Abitibi-Témiscamingue
Processing college 80: Yukon University
Processing college 90: Redeemer University


In [3]:
df = pd.read_csv('University_info.csv')


Data Cleaning

In [4]:
df.head()

,Name,Former names,Type,Established,President,Academic staff,Administrative staff,Students,Address,Campus,...,Director,Founder,Chair,Sports teams,Doctoral students,Mascots,Other names,Accreditation,Owner,Colors
0,Alberta University of the Arts,Alberta College of Art Alberta College of Art ...,Public,1926 ( 1926 ) note 1 1,Daniel Doz 2,145,95,"1,323 3 as of fall 2012","1407 14 Avenue NW Calgary , Alberta T2N 4R3 51...",Urban,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Acadia University,Queen's College (1838–1841) Acadia College (18...,Public university,1838 ; 186 years ago ( 1838 ),Jeffrey J. Hennessy,NaN,"211 full-time, 37 part-time (as of 2008)","4,542 1",NaN,250 acres (101 ha),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Algoma University,NaN,Public,1965,Asima Vezina 2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Athabasca University,NaN,Public university specializing in online dista...,1970,Alex Clark 2,NaN,NaN,"40,630 3",NaN,"Online , rural and urban",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Bishop's University,NaN,Liberal arts college,1843 ; 181 years ago ( 1843 ) 1,NaN,115,NaN,"2,867 1",NaN,"Rural, 220 ha (550 acres) 1 , 550",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.drop(columns=df.columns[21:63], inplace=True)

In [6]:
df.drop(columns=df.columns[2], inplace=True)

In [7]:
df.head()


,Name,Former names,Established,President,Academic staff,Administrative staff,Students,Address,Campus,Colours,...,Website,ASN,Motto,Motto in English,Religious affiliation,Academic affiliations,Endowment,Chancellor,Undergraduates,Colors
0,Alberta University of the Arts,Alberta College of Art Alberta College of Art ...,1926 ( 1926 ) note 1 1,Daniel Doz 2,145,95,"1,323 3 as of fall 2012","1407 14 Avenue NW Calgary , Alberta T2N 4R3 51...",Urban,Red White,...,auarts .ca,54307.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Acadia University,Queen's College (1838–1841) Acadia College (18...,1838 ; 186 years ago ( 1838 ),Jeffrey J. Hennessy,NaN,"211 full-time, 37 part-time (as of 2008)","4,542 1",NaN,250 acres (101 ha),Red and blue,...,www2 .acadiau .ca /home .html,NaN,In pulvere vinces,"""By effort ( lit. ' in dust ' ), you will conq...",Currently non-denominational; initially founde...,"AUCC , IAU , CUSID , CBIE, CUP , Maple League ...",$109.4 million (2021),Bruce Galloway,"4,024",NaN
2,Algoma University,NaN,1965,Asima Vezina 2,NaN,NaN,NaN,NaN,NaN,"Red, Grey, Gold, Light Green",...,algomau .ca,NaN,NaN,NaN,NaN,"COU , Universities Canada",NaN,Mario Turco 1,NaN,NaN
3,Athabasca University,NaN,1970,Alex Clark 2,NaN,NaN,"40,630 3",NaN,"Online , rural and urban",Blue and orange,...,www .athabascau .ca,NaN,Learning for Life 1,NaN,NaN,NaN,NaN,NaN,"35,520 3",NaN
4,Bishop's University,NaN,1843 ; 181 years ago ( 1843 ) 1,NaN,115,NaN,"2,867 1",NaN,"Rural, 220 ha (550 acres) 1 , 550",Purple and silver,...,www .ubishops .ca,NaN,Recti cultus pectora roborant ( Latin ),Righteous ways makes strong the heart,NaN,"AUCC , IAU , QSSF , CBIE, CUP , Maple League o...",C$ 32.5 million 2,Daniel Fournier,"2,340 full-time students 219 part-time students 1",NaN


In [8]:
import re
def clean_text(text):
    if isinstance(text, str):
        text = re.sub(r'\n', ' ', text)  # Remove newlines
        text = re.sub(r'\[', '', text)  # Remove opening square brackets
        text = re.sub(r'\]', '', text)  # Remove closing square brackets
        text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
        text = text.replace('\ufeff', '')  # Remove unicode character
        text = text.replace(', ,', ',')  # Remove extra commas
    return text

# Function to clean and format the Address field
def clean_address(text):
    if isinstance(text, str):
        text = re.sub(r'\d{2,3}°\d{2}′\d{2}″[N|S|E|W]', '', text)  # Remove latitude and longitude
        text = re.sub(r'\(.*?\)', '', text)  # Remove content within parentheses
        text = text.replace(' / ', '').strip()  # Remove ' / ' and strip spaces
    return text

# Read the CSV file

# Apply cleaning function to all fields
df = df.applymap(clean_text)

# Apply address-specific cleaning to the Address field
df['Address'] = df['Address'].apply(clean_address)

# Display the cleaned DataFrame
print(df)

# Optionally save the cleaned data back to CSV
df.to_csv('cleaned_University_info.csv', index=False)

                                  Name  \
0       Alberta University of the Arts   
1                    Acadia University   
2                    Algoma University   
3                 Athabasca University   
4                  Bishop's University   
..                                 ...   
95                  Tyndale University   
96              University Canada West   
97           University of Fredericton   
98  University of Niagara Falls Canada   
99                Yorkville University   

                                         Former names  \
0   Alberta College of Art Alberta College of Art ...   
1   Queen's College (1838–1841) Acadia College (18...   
2                                                 NaN   
3                                                 NaN   
4                                                 NaN   
..                                                ...   
95                                                NaN   
96                                     

In [9]:
import requests
from bs4 import BeautifulSoup as bs
import re
import pandas as pd
def get_info_box2(url):
    # Send a request to the URL
    response = requests.get(url)

    # Parse the content using BeautifulSoup
    soup = bs(response.content, 'html.parser')

    # Find the infobox table with class 'infobox vcard'
    info_box = soup.find('table', {'class': 'infobox vcard'})

    # Dictionary to hold the extracted information
    info = {}

    if info_box:
        # Extract rows from the infobox table
        rows = info_box.find_all('tr')

        for row in rows:
            # Get the header (th) and data (td) from each row
            header = row.find('th')
            data = row.find('td')

            if header and data:
                # Get text from header and data, stripping extra spaces
                key = header.get_text(" ", strip=True)
                value = data.get_text(" ", strip=True)

                # Clean the value
                value = value.replace('\n', ' ')
                value = re.sub(r'\[.*?\]', '', value)  # Remove content within brackets
                value = re.sub(r'\s+', ' ', value).strip()  # Remove extra spaces

                # Store the cleaned key-value pair in the dictionary
                info[key] = value

    return info

# Example usage
url = 'https://en.wikipedia.org/wiki/ABM_College'
college_info = get_info_box(url)
print(college_info)

{'Type': 'Private', 'Established': '2011', 'President': 'Dr. Mohammed Baten', 'Administrative staff': '80', 'Address': '112 - 28 Street SE, #200 , Calgary , Alberta , Canada 51°03′08″N 113°59′34″W  /  51.05218°N 113.99288°W  / 51.05218; -113.99288', 'Colours': 'Red , white', 'Website': 'https://www.abmcollege.com/'}


In [10]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re

def clean_text(text):
    if isinstance(text, str):
        text = re.sub(r'\n', ' ', text)  # Remove newlines
        text = re.sub(r'\[', '', text)  # Remove opening square brackets
        text = re.sub(r'\]', '', text)  # Remove closing square brackets
        text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
        text = text.replace('\ufeff', '')  # Remove unicode character
        text = text.replace(', ,', ',')  # Remove extra commas
    return text

def clean_address(text):
    if isinstance(text, str):
        text = re.sub(r'\d{2,3}°\d{2}′\d{2}″[N|S|E|W]', '', text)  # Remove latitude and longitude
        text = re.sub(r'\(.*?\)', '', text)  # Remove content within parentheses
        text = text.replace(' / ', '').strip()  # Remove ' / ' and strip spaces
    return text

def get_info_box(url):
    # Send a request to the URL
    response = requests.get(url)
    print(f'Fetching URL: {url}')

    # Parse the content using BeautifulSoup
    soup = bs(response.content, 'html.parser')

    # Find the infobox table with class 'infobox vcard'
    info_box = soup.find('table', {'class': 'infobox vcard'})

    # Dictionary to hold the extracted information
    info = {}

    if info_box:
        # Extract rows from the infobox table
        rows = info_box.find_all('tr')

        for row in rows:
            # Get the header (th) and data (td) from each row
            header = row.find('th')
            data = row.find('td')

            if header and data:
                # Get text from header and data, stripping extra spaces
                key = header.get_text(" ", strip=True)
                value = data.get_text(" ", strip=True)

                # Clean the value
                value = clean_text(value)

                # Store the cleaned key-value pair in the dictionary
                info[key] = value

    return info

def save_data_to_csv(title, data):
    df = pd.DataFrame(data)
    # Ensure the 'name' column is the first column
    columns = ['Name'] + [col for col in df.columns if col != 'Name']
    df = df[columns]
    df.to_csv(title, index=False, encoding='utf-8')

# Fetch the webpage
response = requests.get("https://en.wikipedia.org/wiki/List_of_colleges_in_Canada")
soup = bs(response.content, 'html.parser')

# Select all <a> tags within <li> tags under "mw-parser-output" div
college_links = soup.select("div.mw-parser-output ul li a")

# Base URL for Wikipedia
base_url = 'https://en.wikipedia.org'

# List to store college information
college_info_list = []

# Loop through the college links
for index, college in enumerate(college_links):
    # Debugging line to break after 10 iterations, uncomment if needed
    # if index == 10:
    #     break

    # Print progress every 10 colleges
    if index % 10 == 0:
        print(f'Processing college {index}: {college.get_text()}')

    try:
        # Construct the full URL for each college
        relative_url = college['href']
        full_url = base_url + relative_url
        title = college.get_text(strip=True)

        # Retrieve information box for each college and append to the list
        college_info = get_info_box(full_url)
        college_info['Name'] = title
        college_info_list.append(college_info)

    except Exception as e:
        # Print the college name and the error message
        print(f'Error processing college {college.get_text()}: {e}')

# Save the college information to a CSV file
save_data_to_csv('college_info.csv', college_info_list)




Processing college 0: ABM College of Health and Technology
Fetching URL: https://en.wikipedia.org/wiki/ABM_College_of_Health_and_Technology
Fetching URL: https://en.wikipedia.org/wiki/Ambrose_University_College
Fetching URL: https://en.wikipedia.org/wiki/Bay_River_College
Fetching URL: https://en.wikipedia.org/wiki/Bow_Valley_College
Fetching URL: https://en.wikipedia.org/wiki/Grande_Prairie_Regional_College
Fetching URL: https://en.wikipedia.org/wiki/Keyano_College
Fetching URL: https://en.wikipedia.org/wiki/King%27s_University_College_(Edmonton)
Fetching URL: https://en.wikipedia.org/wiki/Lakeland_College_(Alberta)
Fetching URL: https://en.wikipedia.org/wiki/Lethbridge_College
Fetching URL: https://en.wikipedia.org/wiki/MaKami_College
Processing college 10: Medicine Hat College
Fetching URL: https://en.wikipedia.org/wiki/Medicine_Hat_College
Fetching URL: https://en.wikipedia.org/wiki/NorQuest_College
Fetching URL: https://en.wikipedia.org/wiki/Northern_Alberta_Institute_of_Technolog

In [11]:
df = pd.read_csv('college_info.csv')

In [12]:
df.head()

,Name,Type,Established,President,Administrative staff,Address,Colours,Website,Former names,Religious affiliation,...,Education system,President & Chief Executive Officer,Chief Executive Officer,Partners,Formation,Legal status,Purpose,Headquarters,Region served,Official language
0,ABM College of Health and Technology,Private,2011,Dr. Mohammed Baten,80,"112 - 28 Street SE, #200 , Calgary , Alberta ,...","Red , white",https://www.abmcollege.com/,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Ambrose University College,Private,1921,Bryce Ashlin-Mayo,NaN,NaN,"Gold, white, black",ambrose .edu,"Calgary Bible Institute, Alberta School of Eva...","Christian and Missionary Alliance , Church of ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Bay River College,Private,2014,Arshad Mahmood,30,NaN,"Blue , Light Blue",https://bayrivercolleges.ca,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Bow Valley College,Public College (Canada),1965,Dr. Misheck Mwaba 1,500,NaN,Blue,bowvalleycollege .ca,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Grande Prairie Regional College,Polytechnic,1966,Dr. Vanessa Sheane,NaN,NaN,"Dark blue , light blue, pink and yellow 1",www .nwpolytech .ca,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df.drop(columns=df.columns[33:91], inplace=True)
df.drop(columns=df.columns[27], inplace=True)

In [14]:
df.head()

,Name,Type,Established,President,Administrative staff,Address,Colours,Website,Former names,Religious affiliation,...,Affiliation,Endowment,Sporting affiliations,Director,Operational,Provost,Undergraduates,Postgraduates,Other students,Official language
0,ABM College of Health and Technology,Private,2011,Dr. Mohammed Baten,80,"112 - 28 Street SE, #200 , Calgary , Alberta ,...","Red , white",https://www.abmcollege.com/,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Ambrose University College,Private,1921,Bryce Ashlin-Mayo,NaN,NaN,"Gold, white, black",ambrose .edu,"Calgary Bible Institute, Alberta School of Eva...","Christian and Missionary Alliance , Church of ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Bay River College,Private,2014,Arshad Mahmood,30,NaN,"Blue , Light Blue",https://bayrivercolleges.ca,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Bow Valley College,Public College (Canada),1965,Dr. Misheck Mwaba 1,500,NaN,Blue,bowvalleycollege .ca,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Grande Prairie Regional College,Polytechnic,1966,Dr. Vanessa Sheane,NaN,NaN,"Dark blue , light blue, pink and yellow 1",www .nwpolytech .ca,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
import re
def clean_text(text):
    if isinstance(text, str):
        text = re.sub(r'\n', ' ', text)  # Remove newlines
        text = re.sub(r'\[', '', text)  # Remove opening square brackets
        text = re.sub(r'\]', '', text)  # Remove closing square brackets
        text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
        text = text.replace('\ufeff', '')  # Remove unicode character
        text = text.replace(', ,', ',')  # Remove extra commas
    return text

# Function to clean and format the Address field
def clean_address(text):
    if isinstance(text, str):
        text = re.sub(r'\d{2,3}°\d{2}′\d{2}″[N|S|E|W]', '', text)  # Remove latitude and longitude
        text = re.sub(r'\(.*?\)', '', text)  # Remove content within parentheses
        text = text.replace(' / ', '').strip()  # Remove ' / ' and strip spaces
    return text

# Read the CSV file

# Apply cleaning function to all fields
df = df.applymap(clean_text)

# Apply address-specific cleaning to the Address field
df['Address'] = df['Address'].apply(clean_address)

# Display the cleaned DataFrame
print(df)

# Optionally save the cleaned data back to CSV
df.to_csv('cleaned_college_info.csv', index=False)

                                     Name                     Type  \
0    ABM College of Health and Technology                  Private   
1              Ambrose University College                  Private   
2                       Bay River College                  Private   
3                      Bow Valley College  Public College (Canada)   
4         Grande Prairie Regional College              Polytechnic   
..                                    ...                      ...   
269                 Northwest Territories                      NaN   
270                               Nunavut                      NaN   
271                                 Yukon                      NaN   
272                              Category                      NaN   
273                         Canada portal                      NaN   

    Established            President Administrative staff  \
0          2011   Dr. Mohammed Baten                   80   
1          1921    Bryce Ashlin-Mayo 

In [16]:
# Read the CSV files into DataFrames
universities_df = pd.read_csv('cleaned_University_info.csv')
colleges_df = pd.read_csv('cleaned_college_info.csv')

# Concatenate the DataFrames
combined_df = pd.concat([universities_df, colleges_df], ignore_index=True)
combined_df.drop(columns=df.columns[27:38], inplace=True)
# Save the combined DataFrame to a new CSV file
combined_df.to_csv('List_of_colleges_in_Canada.csv', index=False, encoding='utf-8')

# Print the first few rows of the combined DataFrame to verify
print(combined_df.head())

                             Name  \
0  Alberta University of the Arts   
1               Acadia University   
2               Algoma University   
3            Athabasca University   
4             Bishop's University   

                                        Former names  \
0  Alberta College of Art Alberta College of Art ...   
1  Queen's College (1838–1841) Acadia College (18...   
2                                                NaN   
3                                                NaN   
4                                                NaN   

                       Established            President Academic staff  \
0           1926 ( 1926 ) note 1 1         Daniel Doz 2            145   
1    1838 ; 186 years ago ( 1838 )  Jeffrey J. Hennessy            NaN   
2                             1965       Asima Vezina 2            NaN   
3                             1970         Alex Clark 2            NaN   
4  1843 ; 181 years ago ( 1843 ) 1                  NaN            115